<center>
<img src="https://image.aladin.co.kr/product/22964/73/cover500/k532637497_1.jpg" width="200" height="200"><br>
</center>

# 3.2 비트코인 시세 예측

비트코인의 시세를 예측해보자. 비트코인 시세처럼 연속적인 시간에 따라 다르게 측정되는 데이터를 시계열 데이터라 하며, 이를 분석하는것을 시계열 데이터 분석이라고 한다.

### Step 1 탐색: 시간정보가 포함된 데이터 살펴보기

우선 비트코인 데이터를 탐색해보자. 다음 코드로 데이터셋을 살펴보면, 비트코인 데이터는 총 365개의 행으로 이루어진 것을 알 수 있다. 또한 데이터는 결측값 없이 날짜와 가격에 대한 2개의 피쳐로 구성되어 있다.